In [42]:
import glob

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import plotly.express as px
import plotly.graph_objects as go

In [87]:
columns=['approach', 'sort', 'world_size', 'construct', 'threads', 'size', 'duration']
results = pd.read_csv('../results/final/mpi_sort_results.csv', names=columns)

In [88]:
results

,approach,sort,world_size,construct,threads,size,duration
0,merge,enumeration_sort,16,single,1,10000,0.013800
1,partition,enumeration_sort,16,single,1,10000,0.033126
2,merge,merge_sort,16,single,1,10000,0.001347
3,partition,merge_sort,16,single,1,10000,0.001445
4,merge,quick_sort,16,single,1,10000,0.001191
...,...,...,...,...,...,...,...
1291,partition,quick_sort,2,tasks,8,100000000,15.529089
1292,merge,merge_sort,2,tasks,8,100000000,14.271214
1293,partition,merge_sort,2,tasks,8,100000000,14.898220
1294,merge,quick_sort,2,tasks,8,100000000,15.227301


In [89]:
for col in results.columns:
    if col != 'duration':
        print(f'{col}: {results[col].unique()}')

approach: ['merge' 'partition' 'serial']
sort: ['enumeration_sort' 'merge_sort' 'quick_sort']
world_size: [16  8  4  1  2]
construct: ['single' 'parallel' 'tasks']
threads: [1 2 4 8]
size: [    10000    100000   1000000  10000000 100000000]


In [90]:
df = (
    results
    .groupby(['approach', 'sort', 'world_size', 'construct', 'threads', 'size'])['duration'].mean()
    .to_frame('duration')
    .reset_index()
)
df

,approach,sort,world_size,construct,threads,size,duration
0,merge,enumeration_sort,2,parallel,2,10000,0.066418
1,merge,enumeration_sort,2,parallel,2,100000,6.140236
2,merge,enumeration_sort,2,parallel,4,10000,0.035189
3,merge,enumeration_sort,2,parallel,4,100000,3.110686
4,merge,enumeration_sort,2,parallel,8,10000,0.021299
...,...,...,...,...,...,...,...
427,serial,quick_sort,1,tasks,8,10000,0.001542
428,serial,quick_sort,1,tasks,8,100000,0.018815
429,serial,quick_sort,1,tasks,8,1000000,0.220739
430,serial,quick_sort,1,tasks,8,10000000,2.501303


In [96]:
pd.set_option('display.max_rows', None)

In [288]:
summary_table = (
    df
    .loc[df['size'] == 100000]
    .drop(columns=['construct'])
    .pivot(index=['world_size','threads','size'], columns=['approach','sort'])
    ['duration']
)

parallel = summary_table.loc[summary_table.index.get_level_values('world_size') != 1, ['merge','partition']]
serial = summary_table.loc[summary_table.index.get_level_values('world_size') == 1, ['serial']].reset_index('world_size', drop=True)

In [291]:
parallel_df = (
    parallel.loc[:, (slice(None), ('merge_sort', 'quick_sort'))]
    .reset_index()
    .melt(id_vars=['world_size', 'threads', 'size'], value_name='duration')
)

In [301]:
fig = px.line(
    parallel_df.loc[(parallel_df['approach'] == 'merge')],
    x='world_size', y='duration', color='threads',
    facet_col='sort', log_x=True,
    title='Performance Comparison of MPI Merge: Processes vs. Runtime (Array Size = 100000)'
)


fig.show()

In [302]:
fig = px.line(
    parallel_df.loc[(parallel_df['approach'] == 'partition')],
    x='world_size', y='duration', color='threads',
    facet_col='sort', log_x=True,
    title='Performance Comparison of MPI Partition: Processes vs. Runtime (Array Size = 100000)'
)


fig.show()

In [233]:
display(serial, parallel)

approach                    serial                      
sort              enumeration_sort merge_sort quick_sort
threads size                                            
1       10000               0.2368     0.0013     0.0015
        100000             24.0071     0.0137     0.0158
        1000000                nan     0.1674     0.2033
        10000000               nan     1.9961     2.2887
        100000000              nan    22.7555    26.6629
2       10000               0.2453     0.0009     0.0015
        100000             24.3789     0.0111     0.0185
        1000000                nan     0.1216     0.2215
        10000000               nan     1.2654     2.4834
        100000000              nan    13.8979    28.5926
4       10000               0.1249     0.0009     0.0016
        100000             12.2966     0.0083     0.0186
        1000000                nan     0.0899     0.2200
        10000000               nan     0.8333     2.4976
        100000000              nan     9.4784    28.6804
8       10000               0.0693     0.0006     0.0015
        100000              6.2180     0.0065     0.0188
        1000000                nan     0.0757     0.2207
        10000000               nan     0.6009     2.5013
        100000000              nan     5.9809    28.8251

approach                                merge                        \
sort                         enumeration_sort merge_sort quick_sort   
world_size threads size                                               
2          1       10000               0.0600     0.0009     0.0009   
                   100000              6.0387     0.0083     0.0089   
                   1000000                nan     0.0930     0.1114   
                   10000000               nan     1.0982     1.2184   
                   100000000              nan    12.4001    14.1110   
           2       10000               0.0664     0.0012     0.0010   
                   100000              6.1402     0.0103     0.0098   
                   1000000                nan     0.1090     0.1201   
                   10000000               nan     1.2773     1.3176   
                   100000000              nan    14.2537    15.1680   
           4       10000               0.0352     0.0010     0.0010   
                   100000              3.1107     0.0104     0.0097   
                   1000000                nan     0.1097     0.1192   
                   10000000               nan     1.2890     1.3228   
                   100000000              nan    14.1806    15.0895   
           8       10000               0.0213     0.0011     0.0010   
                   100000              1.8421     0.0106     0.0099   
                   1000000                nan     0.1109     0.1188   
                   10000000               nan     1.2996     1.3281   
                   100000000              nan    14.2542    15.1598   
4          1       10000               0.0220     0.0008     0.0007   
                   100000              1.5935     0.0055     0.0057   
                   1000000                nan     0.0592     0.0650   
                   10000000               nan     0.6496     0.7002   
                   100000000              nan     7.2303     8.0031   
           2       10000               0.0188     0.0008     0.0007   
                   100000              1.5640     0.0060     0.0059   
                   1000000                nan     0.0649     0.0681   
                   10000000               nan     0.7526     0.7648   
                   100000000              nan     8.0879     8.4846   
           4       10000               0.0140     0.0008     0.0007   
                   100000              0.9206     0.0062     0.0057   
                   1000000                nan     0.0647     0.0680   
                   10000000               nan     0.7530     0.7571   
                   100000000              nan     8.0751     8.4566   
           8       10000               0.0122     0.0010     0.0008   
                   100000              0.7435     0.0071     0.0065   
                   1000000                nan     0.0651     0.0701   
                   10000000               nan     0.7483     0.7560   
                   100000000              nan     8.0347     8.4405   
8          1       10000               0.0069     0.0009     0.0008   
                   100000              0.4365     0.0050     0.0046   
                   1000000                nan     0.0433     0.0414   
                   10000000               nan     0.4510     0.4484   
                   100000000              nan     4.6336     4.8485   
           2       10000               0.0073     0.0009     0.0008   
                   100000              0.4813     0.0044     0.0039   
                   1000000                nan     0.0442     0.0442   
                   10000000               nan     0.4725     0.4809   
                   100000000              nan     5.0233     5.1471   
           4       10000               0.0063     0.0009     0.0008   
                   100000              0.3837     0.0049     0.0046   
                   1000000                nan     0.0427     0.0441   
                   10000

In [219]:
serial_df = (
    serial
    .reset_index('world_size', drop=True)
    .reset_index()
    .melt(id_vars=['threads'], value_name='duration')
)

serial_df.drop(columns='approach', inplace=True)
serial_df.sort = serial_df.sort.str.replace('_sort', '')
serial_df = serial_df.rename(columns={'duration': 'Duration (s)', 'threads': 'OpenMP Threads', 'sort': 'Sorting Algorithm'})
serial_df

,OpenMP Threads,Sorting Algorithm,Duration (s)
0,1,enumeration,24.0071
1,2,enumeration,24.3789
2,4,enumeration,12.2966
3,8,enumeration,6.2180
4,1,merge,0.0137
5,2,merge,0.0111
6,4,merge,0.0083
7,8,merge,0.0065
8,1,quick,0.0158
9,2,quick,0.0185


### IO Experiments

In [144]:
io = pd.concat([
    pd.read_csv(
        f'../results/final/{approach}_io_results.csv',
        names=['io', 'approach', 'world_size', 'size', 'duration']
    ) for approach in ('serial', 'parallel')
])

In [165]:
pd.options.display.float_format = '{:,.4f}'.format

In [166]:
io_summary = (
    io.drop(columns='approach')
    .sort_values(['io', 'world_size'])
    .pivot(index=['size'], columns=['io', 'world_size'])
    ['duration']
)

display(io_summary)

io           read                                     write                \
world_size     1      2      4      8      16     24     1      2      4    
size                                                                        
1000       0.0000 0.0002 0.0004 0.0006 0.0013 0.1044 0.0001 0.0013 0.0018   
10000      0.0000 0.0002 0.0003 0.0005 0.0009 0.0505 0.0001 0.0002 0.0004   
100000     0.0003 0.0004 0.0004 0.0006 0.0010 0.0459 0.0005 0.0005 0.0009   
1000000    0.0036 0.0025 0.0015 0.0014 0.0016 0.0609 0.0075 0.0035 0.0051   
10000000   0.0378 0.0206 0.0127 0.0098 0.0080 0.0442 0.0860 0.0303 0.0293   
100000000  0.3518 0.2037 0.1122 0.0843 0.0694 0.1439 0.9280 0.2622 0.2761   
1000000000 3.0032 1.7220 0.9151 0.6454 0.5662 0.5831 7.2551 2.3322 2.5684   

io                               
world_size     8      16     24  
size                             
1000       0.0032 0.0032 0.0832  
10000      0.0006 0.0010 0.0587  
100000     0.0009 0.0018 0.0612  
1000000    0.0048 0.0053 0.0895  
10000000   0.0355 0.0374 0.1366  
100000000  0.2923 0.3130 0.4672  
1000000000 2.5032 2.8186 3.2280

In [ ]:

    facet_col_spacing=0.05,
    facet_row='Construct',
    template='seaborn',
    log_x=True,
#     log_y=True,
    title=f'Algorithm Runtime Comparisons in C / OpenMP ({if_cutoff})',
    height=600,
    labels={
        "Duration": "Duration (s)",
        "Size": "Array Size",
     },

In [186]:
px.line(
    (
        io_summary['read']
        .reset_index()
        .melt(id_vars='size', value_name='duration')
        .rename(columns={'world_size': 'World Size'})
    ),
    x='size', y='duration', color='World Size',
    height=500, width=700, template='seaborn',
    log_x=True, #log_y=True,
    labels={'duration': 'Duration (s)', 'size': 'Array Size'},
    title='Duration for IO Read Time vs. Array Size'
)
    

In [187]:
px.line(
    (
        io_summary['write']
        .reset_index()
        .melt(id_vars='size', value_name='duration')
        .rename(columns={'world_size': 'World Size'})
    ),
    x='size', y='duration', color='World Size',
    height=500, width=700, template='seaborn',
    log_x=True, #log_y=True,
    labels={'duration': 'Duration (s)', 'size': 'Array Size'},
    title='Duration for IO Write Time vs. Array Size'
)
    

In [210]:
fig = px.line(
    (
        io_summary
        .reset_index()
        .melt(id_vars='size', value_name='duration')
        .rename(columns={'world_size': 'World Size', 'io': "IO"})
    ),
    x='size', y='duration', color='World Size',
    facet_col='IO', facet_col_spacing=0.03,
    height=450, width=900, template='seaborn',
    log_x=True, #log_y=True,
    labels={'duration': 'Duration (s)', 'size': 'Array Size'},
    title='Performance Chart for IO Time vs. Array Size'
)

# plots for sections/tasks for quick/merge sort
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True, col=2) # assuming second facet
fig.show()